# Imports

In [ ]:
import sys
import os
from google.colab import drive
from google.colab import files
from dotenv import load_dotenv
import json
import warnings
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
%matplotlib inline

# Bootstrap

In [ ]:
np.random.seed(31071967)

# Find and load the .env file from the current or parent directories
load_dotenv()

drive.mount('/content/drive')

with open(f"{os.getenv('PROJECT_PATH')}/src/config.json", 'r') as f:
    project_config = json.load(f)
    project_config.pop('_comment', None)
    project_config.pop('_note', None)
    f.close()

In [ ]:
project_config['STOCKS'] = project_config['STOCKS'].split()
project_config['TKL'] = project_config['STOCKS'][0]
project_config['TKL']

# Run prediciton

In [ ]:
def make_datasets(df, X_cols, y_col):

    LOOK_BACK_DAYS = int(project_config['LOOK_BACK_DAYS'])

    X, y = [], []

    for i in range(LOOK_BACK_DAYS, len(df)):
        X.append(df.loc[i-LOOK_BACK_DAYS:i-1, X_cols].values)
        y.append(df.loc[i, y_col].values[0])

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    dates = pd.to_datetime(df['Date'])
    dates_test = dates[-len(y_test):].values

    return X_train, X_test, y_train, y_test, dates_test

In [ ]:
def predict_next_days(df, model, model_name, model_features, future_days=10):

  X_train, X_test, y_train, y_test, dates_test = make_datasets(df, model_features, 'y_next')

  last_historical_block = X_test[-1:] # Select the last sequence from X_test
  last_historical_day = df.iloc[-1].Date
  future_rolling_block = last_historical_block.copy()
  future_predictions = []

  for _ in range(future_days):

    next_pred_day = model.predict(future_rolling_block)[0]
    future_predictions.append(next_pred_day)

    new_input_row = future_rolling_block[0, -1, :].copy() # Copy last row of features
    new_input_row[0] = next_pred_day # Update the 'y' feature (first feature)
    future_rolling_block = np.roll(future_rolling_block, -1, axis=1) # Shift time window
    future_rolling_block[0, -1, :] = new_input_row # Place the new feature vector at the end

    min_y_next_orig = df_orig['y_next_orig'].min()
    max_y_next_orig = df_orig['y_next_orig'].max()
    np.array(future_predictions) * (max_y_next_orig - min_y_next_orig) + min_y_next_orig

  #print(f"Using {winning_model_name} and {winning_model_features} to predicti the next {future_days} days:")
  #print(f"-------------------------------------------")
  for i, p in enumerate(unscaled_future_predictions, start=1):
    # Convert last_historical_day to datetime object and add days
    prediction_date = pd.to_datetime(last_historical_day) + pd.offsets.BDay(i)
    print(f"{prediction_date.strftime('%Y-%m-%d')}: {float(p):.2f}")

In [ ]:
def make_datasets(df, X_cols, y_col):

    LOOK_BACK_DAYS = int(project_config['LOOK_BACK_DAYS'])

    X, y = [], []

    for i in range(LOOK_BACK_DAYS, len(df)):
        X.append(df.loc[i-LOOK_BACK_DAYS:i-1, X_cols].values)
        y.append(df.loc[i, y_col]) # Corrected line

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    dates = pd.to_datetime(df['Date'])
    dates_test = dates[-len(y_test):].values

    return X_train, X_test, y_train, y_test, dates_test

def predict_next_days(df, model, model_name, model_features, future_days=10):

  X_train, X_test, y_train, y_test, dates_test = make_datasets(df, model_features, 'y_next')

  last_historical_block = X_test[-1:] # Select the last sequence from X_test
  last_historical_day = df.iloc[-1].Date
  future_rolling_block = last_historical_block.copy()
  future_predictions = []

  for _ in range(future_days):

    next_pred_day = model.predict(future_rolling_block)[0]
    future_predictions.append(next_pred_day)

    new_input_row = future_rolling_block[0, -1, :].copy() # Copy last row of features
    new_input_row[0] = next_pred_day # Update the 'y' feature (first feature)
    future_rolling_block = np.roll(future_rolling_block, -1, axis=1) # Shift time window
    future_rolling_block[0, -1, :] = new_input_row # Place the new feature vector at the end

    min_y_next_orig = df_orig['y_next_orig'].min()
    max_y_next_orig = df_orig['y_next_orig'].max()
    # Re-scale the predicted values
    unscaled_future_predictions = np.array(future_predictions) * (max_y_next_orig - min_y_next_orig) + min_y_next_orig

  #print(f"Using {winning_model_name} and {winning_model_features} to predicti the next {future_days} days:")
  #print(f"-------------------------------------------")
  for i, p in enumerate(unscaled_future_predictions, start=1):
    # Convert last_historical_day to datetime object and add days
    prediction_date = pd.to_datetime(last_historical_day) + pd.offsets.BDay(i)
    print(f"{prediction_date.strftime('%Y-%m-%d')}: {float(p):.2f}")


from pathlib import Path

pickles_path = Path(f"{os.getenv('PROJECT_PATH')}{project_config['pickles_directory']}")
folder = Path(pickles_path)


if list(folder.glob(f"{project_config['TKL']}.model*.keras")) != []:

  import ast
  model_path = list(folder.glob(f"{project_config['TKL']}.model*.keras"))[0]
  df_path = list(folder.glob(f"{project_config['TKL']}.df.pkl"))[0]
  df_orig_path = list(folder.glob(f"{project_config['TKL']}.df_orig.pkl"))[0]

  fname = model_path.name  # extract filename only
  base = fname.removesuffix(".keras")
  tkl_name, _, best_model_name, features_str = base.split(".", maxsplit=3)
  best_model_features = ast.literal_eval(features_str)

  print(f"TKL: {tkl_name}")
  print(f"Model: {best_model_name}")
  print(f"Features: {best_model_features}")

  import tensorflow as tf
  df = pd.read_pickle(df_path)
  df_orig = pd.read_pickle(df_orig_path)
  best_model = tf.keras.models.load_model(model_path)

  display(df.tail(1), df_orig.tail(1))
  best_model.summary()

  predict_next_days(df, best_model, best_model_name, best_model_features, future_days=10)